In [ ]:
# Cell 1: Install required packages
%pip install requests python-dotenv openai linkedin-api markdown weasyprint jinja2

In [ ]:
# Cell 2: Import libraries and initialize
import os
import json
from datetime import datetime
from dotenv import load_dotenv

# Load environment variables from parent directory
load_dotenv(os.path.join(os.path.dirname(os.getcwd()), ".env"))

# Check for required tokens
openai_key = os.getenv("OPENAI_API_KEY")
linkedin_email = os.getenv("LINKEDIN_EMAIL")
linkedin_password = os.getenv("LINKEDIN_PASSWORD")

print("🔑 Configuration Status:")
print(f"   OpenAI API Key: {'✅ Found' if openai_key else '❌ Not found'}")
print(f"   LinkedIn Email: {'✅ Found' if linkedin_email else '❌ Not found (manual input available)'}")
print(f"   LinkedIn Password: {'✅ Found' if linkedin_password else '❌ Not found (manual input available)'}")

In [ ]:
# Cell 3: LinkedIn Client & Profile Fetcher

class LinkedInClient:
    """Client for fetching LinkedIn profile data."""
    
    def __init__(self, email=None, password=None):
        self.email = email or os.getenv("LINKEDIN_EMAIL")
        self.password = password or os.getenv("LINKEDIN_PASSWORD")
        self.api = None
        self._authenticated = False
    
    def authenticate(self):
        """Authenticate with LinkedIn."""
        if not self.email or not self.password:
            print("⚠️ LinkedIn credentials not provided")
            return False
        
        try:
            from linkedin_api import Linkedin
            self.api = Linkedin(self.email, self.password)
            self._authenticated = True
            print("✅ LinkedIn authentication successful")
            return True
        except Exception as e:
            print(f"⚠️ LinkedIn API authentication failed: {e}")
            return False
    
    def get_profile(self, public_id=None):
        """Fetch LinkedIn profile data."""
        if not self._authenticated:
            return None
        
        try:
            profile = self.api.get_profile(public_id) if public_id else self.api.get_profile()
            print("✅ Profile data fetched successfully")
            return profile
        except Exception as e:
            print(f"❌ Error fetching profile: {e}")
            return None
    
    def get_contact_info(self, public_id=None):
        """Fetch contact information."""
        if not self._authenticated:
            return None
        
        try:
            contact = self.api.get_profile_contact_info(public_id)
            return contact
        except Exception as e:
            print(f"❌ Error fetching contact info: {e}")
            return None


# Try to fetch from LinkedIn
linkedin = LinkedInClient()
profile_data = None

if linkedin.authenticate():
    profile_data = linkedin.get_profile()
    if profile_data:
        contact = linkedin.get_contact_info()
        if contact:
            profile_data.update(contact)
        print(f"\n📋 Profile: {profile_data.get('firstName', '')} {profile_data.get('lastName', '')}")
        print(f"   Headline: {profile_data.get('headline', 'N/A')}")
else:
    print("\n📝 You can enter your profile data manually in the next cell")

In [ ]:
# Cell 4: Manual Profile Input (Edit this cell with your data)
# Skip this cell if LinkedIn fetch was successful

# ⬇️ EDIT YOUR PROFILE DATA BELOW ⬇️

MANUAL_PROFILE = {
    "firstName": "John",
    "lastName": "Doe",
    "headline": "Senior Software Engineer | Full Stack Developer | Cloud Architect",
    "summary": """Experienced software engineer with 10+ years of experience in building scalable web applications 
and cloud infrastructure. Passionate about clean code, DevOps practices, and mentoring junior developers.
Expert in Python, JavaScript/TypeScript, and cloud platforms (AWS, GCP, Azure).""",
    "locationName": "San Francisco, CA",
    "industryName": "Technology",
    "email": "john.doe@email.com",
    "phone": "+1 (555) 123-4567",
    "website": "https://johndoe.dev",
    
    "experience": [
        {
            "title": "Senior Software Engineer",
            "companyName": "Tech Corp",
            "locationName": "San Francisco, CA",
            "description": """Led development of microservices architecture serving 10M+ users
Reduced deployment time by 60% through CI/CD pipeline improvements
Mentored team of 5 junior developers
Architected real-time data processing pipeline handling 1M events/second""",
            "startDate": {"year": 2020, "month": 3},
            "endDate": None  # Current position
        },
        {
            "title": "Software Engineer",
            "companyName": "StartupXYZ",
            "locationName": "San Francisco, CA",
            "description": """Built core product features using React and Node.js
Implemented payment processing system handling $5M+ monthly transactions
Designed and optimized PostgreSQL database schemas""",
            "startDate": {"year": 2017, "month": 6},
            "endDate": {"year": 2020, "month": 2}
        },
        {
            "title": "Junior Developer",
            "companyName": "Agency Inc",
            "locationName": "Los Angeles, CA",
            "description": """Developed client websites and web applications
Collaborated with design team on UI/UX implementations""",
            "startDate": {"year": 2015, "month": 1},
            "endDate": {"year": 2017, "month": 5}
        }
    ],
    
    "education": [
        {
            "schoolName": "University of California, Berkeley",
            "degreeName": "Bachelor of Science",
            "fieldOfStudy": "Computer Science",
            "startDate": {"year": 2011},
            "endDate": {"year": 2015},
            "description": "Dean's List, GPA 3.8"
        }
    ],
    
    "skills": [
        {"name": "Python"},
        {"name": "JavaScript"},
        {"name": "TypeScript"},
        {"name": "React"},
        {"name": "Node.js"},
        {"name": "AWS"},
        {"name": "Docker"},
        {"name": "Kubernetes"},
        {"name": "PostgreSQL"},
        {"name": "MongoDB"},
        {"name": "GraphQL"},
        {"name": "CI/CD"}
    ],
    
    "certifications": [
        {
            "name": "AWS Solutions Architect - Professional",
            "authority": "Amazon Web Services",
            "dateObtained": {"year": 2022, "month": 8}
        },
        {
            "name": "Certified Kubernetes Administrator (CKA)",
            "authority": "CNCF",
            "dateObtained": {"year": 2021, "month": 5}
        }
    ],
    
    "projects": [
        {
            "title": "Open Source CLI Tool",
            "description": "Built a popular CLI tool for developers with 5K+ GitHub stars",
            "url": "https://github.com/johndoe/cli-tool"
        }
    ]
}

# Use manual profile if LinkedIn fetch failed
if not profile_data:
    profile_data = MANUAL_PROFILE
    print("📝 Using manual profile data")
else:
    print("📡 Using LinkedIn profile data")

print(f"\n👤 Profile loaded: {profile_data.get('firstName', '')} {profile_data.get('lastName', '')}")

In [ ]:
# Cell 5: Resume Generator Class

from openai import OpenAI

class ResumeGenerator:
    """Generates professional resumes from profile data."""
    
    def __init__(self, use_ai=True):
        self.use_ai = use_ai
        self.client = None
        
        if use_ai:
            api_key = os.getenv("OPENAI_API_KEY")
            if api_key:
                self.client = OpenAI(api_key=api_key)
                print("✅ OpenAI client initialized for AI enhancement")
            else:
                print("⚠️ No OpenAI API key found - AI enhancement disabled")
                self.use_ai = False
    
    def enhance_text(self, text, context="resume content"):
        """Enhance text using AI."""
        if not self.client or not self.use_ai:
            return text
        
        try:
            response = self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {
                        "role": "system",
                        "content": f"""You are an expert resume writer. Enhance the following {context} 
to be more impactful and professional. Use strong action verbs and quantify achievements 
where possible. Keep it concise. Return only the enhanced text."""
                    },
                    {"role": "user", "content": text}
                ],
                max_tokens=500,
                temperature=0.7
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"⚠️ AI enhancement failed: {e}")
            return text
    
    def format_date(self, date_dict):
        """Format date dictionary to string."""
        if not date_dict:
            return "Present"
        
        year = date_dict.get("year", 0)
        month = date_dict.get("month", 0)
        
        if not year:
            return "Present"
        
        if month:
            return datetime(year, month, 1).strftime("%b %Y")
        return str(year)
    
    def generate_markdown(self, profile, enhance=False):
        """Generate markdown resume from profile."""
        md = []
        
        # Header
        name = f"{profile.get('firstName', '')} {profile.get('lastName', '')}".strip()
        md.append(f"# {name}\n")
        
        # Headline
        if profile.get("headline"):
            md.append(f"**{profile['headline']}**\n")
        
        # Contact
        contact = []
        if profile.get("email"): contact.append(profile["email"])
        if profile.get("phone"): contact.append(profile["phone"])
        if profile.get("locationName"): contact.append(profile["locationName"])
        if profile.get("website"): contact.append(f"[Portfolio]({profile['website']})")
        
        if contact:
            md.append(" | ".join(contact) + "\n")
        
        md.append("---\n")
        
        # Summary
        if profile.get("summary"):
            md.append("## Professional Summary\n")
            summary = profile["summary"]
            if enhance:
                summary = self.enhance_text(summary, "professional summary")
            md.append(f"{summary}\n")
        
        # Experience
        if profile.get("experience"):
            md.append("## Professional Experience\n")
            for exp in profile["experience"]:
                title = exp.get("title", "")
                company = exp.get("companyName", "")
                location = exp.get("locationName", "")
                start = self.format_date(exp.get("startDate"))
                end = self.format_date(exp.get("endDate"))
                
                md.append(f"### {title}")
                md.append(f"**{company}** | {location} | {start} - {end}\n")
                
                desc = exp.get("description", "")
                if desc:
                    if enhance:
                        desc = self.enhance_text(desc, "job accomplishments")
                    # Format as bullets
                    lines = [l.strip() for l in desc.split("\n") if l.strip()]
                    bullets = [f"- {l}" if not l.startswith("-") else l for l in lines]
                    md.append("\n".join(bullets) + "\n")
        
        # Education
        if profile.get("education"):
            md.append("## Education\n")
            for edu in profile["education"]:
                school = edu.get("schoolName", "")
                degree = edu.get("degreeName", "")
                field = edu.get("fieldOfStudy", "")
                year = edu.get("endDate", {}).get("year", "")
                
                degree_str = f"{degree} in {field}" if degree and field else degree or field
                
                md.append(f"### {school}")
                md.append(f"{degree_str} | {year}\n")
                
                if edu.get("description"):
                    md.append(f"{edu['description']}\n")
        
        # Skills
        if profile.get("skills"):
            md.append("## Skills\n")
            skills = [s.get("name", "") for s in profile["skills"] if s.get("name")]
            md.append(", ".join(skills) + "\n")
        
        # Certifications
        if profile.get("certifications"):
            md.append("## Certifications\n")
            for cert in profile["certifications"]:
                name = cert.get("name", "")
                authority = cert.get("authority", "")
                year = cert.get("dateObtained", {}).get("year", "")
                md.append(f"- **{name}** - {authority} ({year})")
            md.append("")
        
        # Projects
        if profile.get("projects"):
            md.append("## Projects\n")
            for proj in profile["projects"]:
                title = proj.get("title", "")
                desc = proj.get("description", "")
                url = proj.get("url", "")
                
                if url:
                    md.append(f"### [{title}]({url})")
                else:
                    md.append(f"### {title}")
                md.append(f"{desc}\n")
        
        return "\n".join(md)
    
    def save_markdown(self, content, filepath="resume.md"):
        """Save resume to markdown file."""
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(content)
        print(f"✅ Markdown resume saved: {filepath}")
        return filepath
    
    def generate_pdf(self, markdown_content, filepath="resume.pdf"):
        """Convert markdown to styled PDF."""
        try:
            import markdown
            from weasyprint import HTML
            
            html_content = markdown.markdown(markdown_content)
            
            styled_html = f"""
            <!DOCTYPE html>
            <html>
            <head>
                <style>
                    @page {{ size: letter; margin: 0.75in; }}
                    body {{
                        font-family: 'Helvetica Neue', Arial, sans-serif;
                        font-size: 10.5pt;
                        line-height: 1.4;
                        color: #333;
                    }}
                    h1 {{
                        font-size: 22pt;
                        color: #1a365d;
                        margin-bottom: 5px;
                        border-bottom: 2px solid #2b6cb0;
                        padding-bottom: 8px;
                    }}
                    h2 {{
                        font-size: 13pt;
                        color: #1a365d;
                        margin-top: 16px;
                        margin-bottom: 8px;
                        border-bottom: 1px solid #cbd5e0;
                        padding-bottom: 4px;
                    }}
                    h3 {{
                        font-size: 11pt;
                        color: #2d3748;
                        margin-bottom: 2px;
                        margin-top: 10px;
                    }}
                    p {{ margin: 4px 0; }}
                    ul {{ margin: 4px 0; padding-left: 18px; }}
                    li {{ margin: 2px 0; }}
                    strong {{ color: #2d3748; }}
                    a {{ color: #2b6cb0; text-decoration: none; }}
                    hr {{ border: none; border-top: 1px solid #e2e8f0; margin: 12px 0; }}
                </style>
            </head>
            <body>{html_content}</body>
            </html>
            """
            
            HTML(string=styled_html).write_pdf(filepath)
            print(f"✅ PDF resume saved: {filepath}")
            return filepath
        except ImportError:
            print("⚠️ PDF generation requires: pip install weasyprint markdown")
            return None
        except Exception as e:
            print(f"❌ PDF generation failed: {e}")
            return None


# Initialize generator
generator = ResumeGenerator(use_ai=True)
print("\n📝 Resume Generator ready!")

In [ ]:
# Cell 6: Generate Resume
# ⬇️ SET OPTIONS BELOW ⬇️

USE_AI_ENHANCEMENT = True  # Set to False for faster generation without AI

print("="*60)
print("🤖 Generating Resume")
print("="*60)

if USE_AI_ENHANCEMENT:
    print("\n✨ AI Enhancement: ENABLED")
    print("   (This may take 30-60 seconds)")
else:
    print("\n⚡ AI Enhancement: DISABLED (fast mode)")

# Generate markdown resume
print("\n📝 Generating markdown...")
resume_md = generator.generate_markdown(profile_data, enhance=USE_AI_ENHANCEMENT)

# Save markdown
generator.save_markdown(resume_md, "resume.md")

# Generate PDF
print("\n📄 Generating PDF...")
generator.generate_pdf(resume_md, "resume.pdf")

# Save profile data for future updates
with open("profile_data.json", "w", encoding="utf-8") as f:
    json.dump(profile_data, f, indent=2)
print("\n💾 Profile data saved to profile_data.json")

print("\n" + "="*60)
print("✅ Resume generation complete!")
print("="*60)
print("\n📂 Generated files:")
print("   • resume.md - Markdown format")
print("   • resume.pdf - PDF format")
print("   • profile_data.json - Profile data (for updates)")

In [ ]:
# Cell 7: Preview Resume (Markdown)
from IPython.display import Markdown, display

print("📄 Resume Preview:\n")
print("="*60)
display(Markdown(resume_md))

In [ ]:
# Cell 8: Update Existing Resume
# Run this cell to update your resume with new changes

# Load existing profile data
if os.path.exists("profile_data.json"):
    with open("profile_data.json", "r", encoding="utf-8") as f:
        profile_data = json.load(f)
    print("✅ Loaded existing profile data")
    
    # You can modify profile_data here before regenerating
    # Example: Add new skill
    # profile_data["skills"].append({"name": "New Skill"})
    
    # Regenerate resume
    resume_md = generator.generate_markdown(profile_data, enhance=False)
    generator.save_markdown(resume_md, "resume.md")
    generator.generate_pdf(resume_md, "resume.pdf")
    
    print("\n✅ Resume updated!")
else:
    print("❌ No profile_data.json found. Run Cell 6 first.")